# Mangrove Gain: 1988-2024
The code in this file creates an HTML file that visualizes the gain in mangrove forest coverage from 1988 to 2024 on an interactive Earth Engine map. It adds individual layers for each year of the mangrove extent, allowing the user to view and compare areas of mangrove loss. 

In [2]:
# --- Run underlying python scripts
import GEN01_GEE_Authenticate
import GEN02_AOI as  aoi_def
import GEN03_helper_functions as HF
import GEN04_mangrove_layers as ML 
import GEN05_HTML_layout as HTML 
import GEN06_shapefiles as shp


In [ ]:
import ee, os

SERVICE_ACCOUNT = os.environ.get("GEE_SERVICE_ACCOUNT")
KEY_FILE = "key.json"   # GitHub Actions writes the secret here

credentials = ee.ServiceAccountCredentials(SERVICE_ACCOUNT, KEY_FILE)
ee.Initialize(credentials)

# ee.Authenticate()
# ee.Initialize(project="gee-mekong-map")


In [4]:
# ------------- Packages & GEE ------------
import ee                                       # Google Earth Engine API for Python
import geemap.foliumap as geemap_folium         # Geemap module integrating Folium maps
import folium                                   # Interactive mapping library based on Leaflet.js
import geemap                                   # Toolkit for working with Google Earth Engine in Python
from geemap import cartoee                      # Module for exporting and visualizing Earth Engine maps using Matplotlib
import ipyleaflet                               # Interactive maps in Jupyter using Leaflet
import ipywidgets as widgets                    # Interactive UI controls for Jupyter notebooks
from branca.element import Element              # Low-level HTML/JS elements for Folium/Branca maps
import geopandas as gpd                         # Spatial data handling (GeoDataFrames)
import glob                                     # File pattern matching (e.g., list all .tif files in a folder)
import json                                     # JSON encoding and decoding (read/write .json files)


In [ ]:
# ------------- Calculate mangrove gain per time interval -------------
# Mangrove gain = mangroves not present earlier but present later
gain_1988_1992 = ML.mangrove_1992.And(ML.mangrove_1988.Not())
gain_1992_1997 = ML.mangrove_1997.And(ML.mangrove_1992.Not())
gain_1997_2001 = ML.mangrove_2001.And(ML.mangrove_1997.Not())
gain_2001_2005 = ML.mangrove_2005.And(ML.mangrove_2001.Not())
gain_2005_2010 = ML.mangrove_2010.And(ML.mangrove_2005.Not())
gain_2010_2015 = ML.mangrove_2015.And(ML.mangrove_2010.Not())
gain_2015_2020 = ML.mangrove_2020.And(ML.mangrove_2015.Not())
gain_2020_2024 = ML.mangrove_2024.And(ML.mangrove_2020.Not())


geemap_folium.ee_initialize = lambda *args, **kwargs: None

# Create a folium-based map
Map = geemap_folium.Map(center=[9.2, 105.75], zoom=11, initialize=False)



Map.add_basemap('SATELLITE')


In [6]:
# ------------- Add gain layers with palette and generate tile URLs -------------
gain_layers = {
    "Mangrove gain (1988-1992)": (gain_1988_1992, ML.color_1988_1992_gain),
    "Mangrove gain (1992-1997)": (gain_1992_1997, ML.color_1992_1997_gain),
    "Mangrove gain (1997-2001)": (gain_1997_2001, ML.color_1997_2001_gain),
    "Mangrove gain (2001-2005)": (gain_2001_2005, ML.color_2001_2005_gain),
    "Mangrove gain (2005-2010)": (gain_2005_2010, ML.color_2005_2010_gain),
    "Mangrove gain (2010-2015)": (gain_2010_2015, ML.color_2010_2015_gain),
    "Mangrove gain (2015-2020)": (gain_2015_2020, ML.color_2015_2020_gain),
    "Mangrove gain (2020-2024)": (gain_2020_2024, ML.color_2020_2024_gain),
}

# -------------- Add gain maps -----------------
Map.addLayer(gain_1988_1992.updateMask(gain_1988_1992), {"palette": [ML.color_1988_1992_gain]}, "Mangrove gain (1988-1992)")
Map.addLayer(gain_1992_1997.updateMask(gain_1992_1997), {"palette": [ML.color_1992_1997_gain]}, "Mangrove gain (1992-1997)")
Map.addLayer(gain_1997_2001.updateMask(gain_1997_2001), {"palette": [ML.color_1997_2001_gain]}, "Mangrove gain (1997-2001)")
Map.addLayer(gain_2001_2005.updateMask(gain_2001_2005), {"palette": [ML.color_2001_2005_gain]}, "Mangrove gain (2001-2005)")
Map.addLayer(gain_2005_2010.updateMask(gain_2005_2010), {"palette": [ML.color_2005_2010_gain]}, "Mangrove gain (2005-2010)")
Map.addLayer(gain_2010_2015.updateMask(gain_2010_2015), {"palette": [ML.color_2010_2015_gain]}, "Mangrove gain (2010-2015)")
Map.addLayer(gain_2015_2020.updateMask(gain_2015_2020), {"palette": [ML.color_2015_2020_gain]}, "Mangrove gain (2015-2020)")
Map.addLayer(gain_2020_2024.updateMask(gain_2020_2024), {"palette": [ML.color_2020_2024_gain]}, "Mangrove gain (2020-2024)")



In [7]:
# ------- Add legend      --------
legend_title = "Mangrove Gain Map (1988-2024)"
HTML.add_folium_legend(Map, legend_title, HTML.legend_dict_mangrove_GAIN, style=HTML.style)

# ------- Add scalebar     -------
Map.add_child(HTML.ScaleBar(font_size="14px"))

# ------- Add north arrow --------
HTML.add_north_arrow(Map, position="bottomleft", arrow_size="35px", text_size="25px")

# ------- Add Living LAb logo -----
Map.get_root().html.add_child(folium.Element(HTML.logo_html))

In [8]:
# Add the commune boundaries to the map
folium.GeoJson(
    shp.geojson_commune,
    name="Commune boundaries",
    style_function=shp.commune_style,
    ).add_to(Map)


#  Add the sea dikes group to the map
shp.sea_dikes_group.add_to(Map)


# Add the breakwater group to the map
shp.breakwaters_group.add_to(Map)

# other human activities
shp.Extra_group.add_to(Map)
folium.LayerControl().add_to(Map)   

In [ ]:
Map.to_html("Mangrove_GAIN_map.html")
